In [1]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sklearn
print(sklearn.__version__)

0.24.2


### 모듈 호츨 

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

### 데이터 구성 확인

In [4]:
import pandas as pd 

iris = load_iris()
# iris.data 는 iris 데이터에서 피처만으로 된 데이터를 numpy로 가지고 있음.
iris_data = iris.data 
# iris.target 은 iris 데이터 세트에서 레이블(결정 값) 데이터를 numpy로 가지고 있음,
iris_label = iris.target
print('iris target 값 : ', iris_label)
print('iris target 명 : ', iris.target_names)

# Df로 변환
iris_df = pd.DataFrame(data = iris_data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df.head()

iris target 값 :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
iris target 명 :  ['setosa' 'versicolor' 'virginica']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


' 피처에는 꽃잎 길이 , 너비 , 꽃받침 길이 ,너비 ,   
label은 0 , 1, 2, 세가지 값  (0 : setosa , 1 : versicolor , 2: virginica)

### 학습 데이터 분리 

In [5]:
X_train,X_test, y_train, y_test = train_test_split(iris_data, iris_label, 
                                                  test_size =0.2 , random_state =11)

### 객체생성

In [6]:
dt_clf = DecisionTreeClassifier(random_state =11)
# 학습 fit()
dt_clf.fit(X_train, y_train)
# 예측 수행
y_hat = dt_clf.predict(X_test)

### 예측성능 평가 

In [7]:
# metrics 모듈의 accuracy_score( 정확도  측정)
from sklearn.metrics import accuracy_score
# y_tset: 실제 데이터 , y_hat : 예측데이터
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test,y_hat)))


예측 정확도 : 0.9333


### Model Selection 모듈 
학습데이터와 검증 데이터 분리 , 교차검증 부할 및 평가   
or Estimator의 하이퍼 파라미터 튜닝 하기 위한 다양한 함수 및 클래스 제공 



#### train_test_split() : 학습데이터 분리 

In [8]:
# 학습데이터 세트로만 학습하고 예측 (분리 x )
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
# 학습
dt_clf.fit(train_data,train_label)
# 예측
pred = dt_clf.predict(train_data)
print(accuracy_score(train_label, pred))

1.0


In [9]:
# 분리 
iris_data = load_iris()
dt_clf = DecisionTreeClassifier()

X_train,X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                  test_size =0.3 , random_state =121)


In [10]:
# 학습 및 예측 
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도 : {0:.4f}'.format(accuracy_score(y_test,pred)))

예측 정확도 : 0.9556


### 교차검증 
과적합 문제 -> 교차검증 : 여러 세트로 구성된 학습데이터와 검증 데이터 세트에서 학습과 평가 수행  
학습데이터를 다시한번 더 분할 
#####  K 폴드 교차검증 : KFold 클래스 , stratifiedKFold 클래스 

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold  # 분할이 kfold 
import numpy as np 

In [12]:
iris = load_iris()
features =iris.data  # X 데이터 셋 
label = iris.target  # Y 분류 레이블 값 
dt_clf = DecisionTreeClassifier(random_state = 156) 

# 5개의 폴드 세트로 분리하는 Kfold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성 
kfold = KFold(n_splits = 5)   # 5개로 분할 
cv_accuracy = []
print('붓꽃 데이터 세트의 크기 :', features.shape[0])

붓꽃 데이터 세트의 크기 : 150


##### 분리
150개 데이터 -> 5 분할 , 120 (학습용) / 30 (검증용), 5번 

In [13]:
n_iter =0 

# KFold 객체의 split()을 호출하면 폴드별 학습용 , 검증용 테스트의 row 인덱스를 array로 반환 
for train_index, test_index in kfold.split(features):
    # 반환된 인덱스를 이용해 학습용 , 검증용 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습 및 예측 
    dt_clf.fit(X_train,y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1 

    # 반복시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차검증 정확도:{1}, 학습데이터 크기 : {2}, 검증데이터 크기: {3}'
         .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증세트 인덱스 : {1}'.format(n_iter, test_index))
    # 검증 데이터 인덱스는 매 번 변함 5 4 3 2 1 
    cv_accuracy.append(accuracy)
    # 세트별 정확도를 리스트에 추가 
    
# 개별 iteration 별 정확도 합하여 평균 정확도 계산 (최종 평가 )
print('\n ## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차검증 정확도:1.0, 학습데이터 크기 : 120, 검증데이터 크기: 30
#1 검증세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차검증 정확도:0.9667, 학습데이터 크기 : 120, 검증데이터 크기: 30
#2 검증세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차검증 정확도:0.8667, 학습데이터 크기 : 120, 검증데이터 크기: 30
#3 검증세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차검증 정확도:0.9333, 학습데이터 크기 : 120, 검증데이터 크기: 30
#4 검증세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차검증 정확도:0.7333, 학습데이터 크기 : 120, 검증데이터 크기: 30
#5 검증세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

 ## 평균 검증 정확도: 0.9


##### Stratified K 폴드 
: 불균형한 분포도를 가진 레이블(결정 클래스) 데이터 집합을 위한 K폴드 방식  
ex) 대출 사기 데이터 (사기 :1, 정상 :0) 사기(1) 의 비율 매우 적겠죠잉  
-> 랜덤하게 학습 및 테스트 분할해도 0과 1의 비율 반영 x 

In [17]:
#Kfold 
import pandas as pd
iris = load_iris()
iris_df= pd.DataFrame(data =iris ,columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

0    50
1    50
2    50
Name: label, dtype: int64

In [22]:
kfold = KFold(n_splits =3 )
n_iter = 0
#for 루프 ,학습 및 테스트 데이터 인덱스 추출
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

## 교차검증 : 1
학습 레이블 데이터 분포: 
 1    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    50
Name: label, dtype: int64
## 교차검증 : 2
학습 레이블 데이터 분포: 
 0    50
2    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    50
Name: label, dtype: int64
## 교차검증 : 3
학습 레이블 데이터 분포: 
 0    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 2    50
Name: label, dtype: int64


##### 교차 검증시 마다 학습레이블과 검증레이블이 완전히 다른값으로 추출 됨 

### stratifiedKFold로 전체 레이블 값의 분포도 반영 

In [25]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits =3)
n_iter = 0 

for train_index, test_index in skf.split(iris_df, iris_df['label']):  # split인자로 피처 데이터 + 레이블 데이터 
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print('## 교차검증 : {0}'.format(n_iter))
    print('학습 레이블 데이터 분포: \n', label_train.value_counts())
    print('검증 레이블 데이터 분포: \n', label_test.value_counts())

## 교차검증 : 1
학습 레이블 데이터 분포: 
 2    34
0    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
1    17
2    16
Name: label, dtype: int64
## 교차검증 : 2
학습 레이블 데이터 분포: 
 1    34
0    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 0    17
2    17
1    16
Name: label, dtype: int64
## 교차검증 : 3
학습 레이블 데이터 분포: 
 0    34
1    33
2    33
Name: label, dtype: int64
검증 레이블 데이터 분포: 
 1    17
2    17
0    16
Name: label, dtype: int64


##### 학습레이블과 검증 레이블 데이터 값의 분포도가 동일하게 할당

In [34]:
# 교차검증 수행 
dt_clf = DecisionTreeClassifier(random_state = 156)

skfold = StratifiedKFold(n_splits=3 )
n_iter = 0 
cv_accuracy =[] # 교차검증 별 정확도 담을 리스트 

for train_index, test_index in skfold.split(features , label):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    # 학습과 예측 
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    # 분할 반복시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차검증 정확도 :{1}, 학습 데이터 크기 : {2}, 검증데이터 크기 : {3}'
         .format(n_iter, accuracy , train_size, test_size))
    print('#{0}, 검증 세트 인덱스 : {1}'.format(n_iter, test_index))
    cv_accuracy.append(accuracy)

# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도 :', np.round(cv_accuracy,4))
print('## 평균 검증 정확도 :', np.mean(cv_accuracy))
    


#1 교차검증 정확도 :0.98, 학습 데이터 크기 : 100, 검증데이터 크기 : 50
#1, 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차검증 정확도 :0.94, 학습 데이터 크기 : 100, 검증데이터 크기 : 50
#2, 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차검증 정확도 :0.98, 학습 데이터 크기 : 100, 검증데이터 크기 : 50
#3, 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도 : [0.98 0.94 0.98]
## 평균 검증 정확도 : 0.9666666666666667


### 교차 검증 간편하게 : cross_val_score() 
##### cross_val_score(estimator, X (features), y(label) , scoring = (성능 평가 지표) , cv( 교차검증 폴드수 ) . . .)

In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris 
import numpy as np

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state =157)

data = iris_data.data
label = iris_data.target

# 성능 평가 지표 acuuracy, 교차 검증 세트 3개 
scores = cross_val_score(dt_clf, data, label, scoring = 'accuracy', cv =3)
print('교차 검증 별 정확도 :', np.round(scores,4))
print('평균 검증 정확도 : ', np.round(np.mean(scores),4))

교차 검증 별 정확도 : [0.98 0.94 0.98]
평균 검증 정확도 :  0.9667


### GridSearchCV : 교차검증 & 최적 하이퍼 파라미터 튜닝 

In [48]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_iris 

# 데이터 로딩 , 데이터 분리 (학습 / 테스트)
iris = load_iris()
X_train, X_test, y_train , y_test = train_test_split(iris_data.data , iris_data.target,
                                                    test_size = 0.2 , random_state= 121)
dtree = DecisionTreeClassifier()    # estimator 객체 

### 파라미터를 딕셔너리 형태로 설정 
parameters = {'max_depth' : [1,2,3], 'min_samples_split' : [2,3] }

In [50]:
import pandas as pd 

# pram_grid의 하이퍼 파라미터를 3개의 train/ test set 폴드로 나누어 테스트 수행 설정 
grid_dtree= GridSearchCV(dtree, param_grid =parameters , cv =3 , refit =True)

# 학습데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습 평가 수행 
grid_dtree.fit(X_train, y_train)
# 결과는 .cv_results_ 속성에 기록됨 

# GridSearchCV의 결과를 추출해 df로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score','split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


### 총 6번 파라미터를 변경하며 학습 및 평가를 수행 
* params : 수행할 때마다 적용된 개별 하이퍼 파라미터 값  
* rank_test_score : 결과 예측 성능 순위  (1 - 최적 하이퍼파라미터)   
* meant_test_score : 개별 하이퍼 파라미터별로 cv의 폴딩 테스트 세트에 대해 총 수행한 평가 평균값  
4,5가 공동 1위 성능 

In [ ]:
print('Gred)